Using spacy <br>
```
conda install -c conda-forge spacy
conda install -c conda-forge cupy
python -m spacy download en_core_web_trf

pip install langchain
pip install pinecone-client
```

Note: 
* Flan T5 XL max length is 512
* Flan T5 XXL max length is 1024

In [46]:
from typing import List
def load_from_txt_file(filename: str) -> List:
    # Open the file in read mode
    with open(filename, "r", encoding='utf-8') as file:
        # Read the contents of the file
        text = file.read()
    # Split the contents of the file into a list of sentences
    sentences = text.splitlines()
    return sentences

textbook = load_from_txt_file('cleaned_yale_patt_pages.txt')
# load_from_txt_file('cleaned_yale_patt_pages.txt')

textbook = ' '.join(textbook)

In [48]:
len(textbook)

1526415

In [1]:
from langchain import text_splitter
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter, NLTKTextSplitter, SpacyTextSplitter

In [52]:
# good examples here: https://langchain.readthedocs.io/en/latest/modules/utils/combine_docs_examples/textsplitter.html

tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')
text_splitter = CharacterTextSplitter(separator='. ').from_huggingface_tokenizer(tokenizer, chunk_size=300, chunk_overlap=0)
texts = text_splitter.split_text(textbook)
print(len(texts))

In [33]:
# todo: check to see if chunks are less than chunk_size

tokenizer = AutoTokenizer.from_pretrained('google/flan-t5-xl')
tokens = tokenizer.encode(texts[22], return_tensors='pt')
print(tokens)
print(tokens.shape)

Token indices sequence length is longer than the specified maximum sequence length for this model (3793 > 512). Running this sequence through the model will result in indexing errors


tensor([[  101,  1597, 30587,  ...,  2530,     5,     1]])


In [ ]:

flan_xl_text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(AutoTokenizer.from_pretrained('google/flan-t5-xl'), chunk_size=300, chunk_overlap=0)
texts = flan_xl_text_splitter.split_text(" ".join(textbook))
# print(texts[0])
# print(len(texts))